In [1]:
import sys; sys.path.append("../")
from feature_extractor import FeatureExtractor
import torch
import numpy as np
import time
from cascade import CascadeClassifier

In [3]:
torch.cuda.is_available()

True

In [4]:
parent = '../hFeatures/'

# Features Extraction

In [5]:

feature_extractor = FeatureExtractor(shape=(19,19), 
                                     percentile=20,
                                     all_features_file=parent + "all_features.npz",
                                     selected_features_file=parent + "selected_features.npz",
                                     labels_file=parent + "labels.npy",
                                     indecies_file=parent + "indecies.npy",)

In [7]:

feature_extractor.extractFeatures(pos_path='../new_data/train/face', 
                                  neg_path='../new_data/train/non-face')
print('done')


In [ ]:
del feature_extractor.f2
del feature_extractor.f3
del feature_extractor.f4

In [ ]:
indecies, X = feature_extractor.selectPercentile()
y = np.load(parent + 'labels.npy')
print(X.shape, y.shape)
print('done')


# Training

In [ ]:
s_t = time.time()
cascadeClassifier= CascadeClassifier(X, y, batchsize=1000, verbose=False, layers=[200])
tr_acc = cascadeClassifier.train()

cascadeClassifier.save(parent + 'cascadeClassifier.pkl')

print('accuracy: ', tr_acc)
print('time taken: ', time.time() - s_t, 's')
print('done')

# Testing

In [ ]:
# accuracy imports
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [ ]:
s_t = time.time()
t_f_idx_map, t_features, t_labels = feature_extractor.extractFeaturesByIndecies(pos_path='face.test/test/face/',
                                            neg_path='face.test/test/non-face/',
                                            cascadeClassifier=cascadeClassifier)
print(t_features.shape, t_labels.shape)
predictions = cascadeClassifier.predict(t_features, t_f_idx_map)
print('test accuracy: ', np.sum(predictions == t_labels) / t_labels.shape[0])
print('time taken: ', time.time() - s_t, 's')

In [ ]:
# report
print('accuracy: ', accuracy_score(t_labels, predictions))
print('f1 score: ', f1_score(t_labels, predictions))
print('precision: ', precision_score(t_labels, predictions))
print('recall: ', recall_score(t_labels, predictions))
print('confusion matrix: \n', confusion_matrix(t_labels, predictions))
print('classification report: \n', classification_report(t_labels, predictions))
